# Ingestão, Tratamento e Alimentação do SQL com os Dados Nuclea

## Ingestão dos Dados

In [ ]:
%%writefile .env
ORACLE_USER=RM567189
ORACLE_PASSWORD="230795"
ORACLE_DSN=oracle.fiap.com.br:1521/orcl

Overwriting .env


In [ ]:
# Carregando as bibliotecas que usaremos nessa parte
!pip install oracledb
from google.colab import drive
import pandas as pd
import numpy as np
import oracledb
import os
from dotenv import load_dotenv
from datetime import datetime
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.2 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
# Criando os DataFrames
load_dotenv()

CREDENTIALS = {
'user' : os.getenv('ORACLE_USER'),
'password' : os.getenv('ORACLE_PASSWORD'),
'dsn' : os.getenv('ORACLE_DSN')
}

df_auxiliar_nuclea = pd.read_csv("/content/drive/MyDrive/FIAP/base_auxiliar_fiap.csv")
df_boletos_nuclea = pd.read_csv("/content/drive/MyDrive/FIAP/base_boletos_fiap.csv")

In [ ]:
# Visualizando o df do base_auxiliar_fiap.csv
df_auxiliar_nuclea.head()

,id_cnpj,cd_cnae_prin,uf,sacado_indice_liquidez_1m,cedente_indice_liquidez_1m,score_materialidade_evolucao,media_atraso_dias,indicador_liquidez_quantitativo_3m,share_vl_inad_pag_bol_6_a_15d,score_quantidade_v2,score_materialidade_v2
0,c137becbf8aeb57d1cb29eafa0506837ae986c18e10ba4...,4645101.0,PE,0.448191,0.677486,776.0,129.786932,0.92,0.03242,979.0,989.0
1,b1bb807c40ce30f95cc5259ad2eec9b9b9296e5b65a7fa...,1052000.0,CE,0.768871,NaN,220.0,90.203666,0.77,0.13992,938.0,989.0
2,4733136b86db508a2c0dcdfd849bbdd1452f30d4cb3ca0...,4781400.0,MS,0.798947,NaN,572.0,165.500000,0.96,0.00000,947.0,988.0
3,982036e318d814e915ae8c0d187cb125a6e8c5fa4b20f5...,4693100.0,PB,0.330366,0.751691,431.0,138.000000,0.87,0.34612,28.0,881.0
4,d63a468b6d43dee9a4c5aa7f8e9dab9a4b5584b5505a00...,2539001.0,SP,0.526669,NaN,173.0,84.619048,0.39,0.15071,74.0,949.0


In [ ]:
# Visualizando o df do base_boletos_fiap.csv
df_boletos_nuclea.head()

,id_boleto,id_pagador,id_beneficiario,dt_emissao,dt_vencimento,dt_pagamento,vlr_nominal,vlr_baixa,tipo_baixa,tipo_especie
0,00c6dec1b869d310010c8e3741262d7e96b0a622a7cf40...,24037765228ca1bce6743570728395541ca481ac79d854...,cf98a6d61719ae21047ea999c57051ffcae0809677068f...,2024-03-27,2024-05-09,2024-03-29,2714.66,NaN,5 - Baixa integral por solicitacao do cedente,DM DUPLICATA MERCANTIL
1,6548008e566d8f6696c5561a845248f61bfd2e3891711c...,24037765228ca1bce6743570728395541ca481ac79d854...,a711fb1183bc159fdcb672d61f9e052cda50f9e0a76e68...,2024-03-13,2024-05-05,2024-05-06,5522.63,5522.63,1 - Baixa integral intrabancaria,DMI DUPLICATA MERCANTIL INDICACAO
2,7409b9a978ccf059c68841c2317a3196adfd8d405854fa...,1c63664185066fb412e0803d751efbde86444efd8c265f...,8b4a7a67986a724fbc3ea10174aef547975c3d7257a3b1...,2024-05-17,2024-05-30,2024-05-27,3338.30,3338.30,0 - Baixa integral interbancaria,DM DUPLICATA MERCANTIL
3,2ef9ddc070a19a5af5d06c5605dc4fa90dc67052e70ffa...,1c63664185066fb412e0803d751efbde86444efd8c265f...,8b4a7a67986a724fbc3ea10174aef547975c3d7257a3b1...,2024-05-16,2024-05-29,2024-05-20,4519.18,4519.18,0 - Baixa integral interbancaria,DM DUPLICATA MERCANTIL
4,59e12a5e65532caf3bdb053f0e6d7af3b10a34b6c69029...,db8c5dc48c47d5ecae33324ee886afa21de700c19d8154...,00ac28a417bb5fe5daeafecd84190e8d794ed125971502...,2024-03-28,2024-05-27,2024-05-27,13416.18,13416.18,0 - Baixa integral interbancaria,DMI DUPLICATA MERCANTIL INDICACAO


## Tratando os Dados

### DataFrame Boletos

In [ ]:
# Verificando os tipos de dados do df de boletos
df_boletos_nuclea.dtypes

,0
id_boleto,object
id_pagador,object
id_beneficiario,object
dt_emissao,object
dt_vencimento,object
dt_pagamento,object
vlr_nominal,float64
vlr_baixa,float64
tipo_baixa,object
tipo_especie,object


In [ ]:
# Transformando os dados dt_emissao, dt_vencimento e dt_pagamento em data
colunas_data = ["dt_emissao", "dt_vencimento", "dt_pagamento"]
for col in colunas_data:
  df_boletos_nuclea[col] = pd.to_datetime(df_boletos_nuclea[col], format = '%Y-%m-%d', errors = "coerce")
df_boletos_nuclea.dtypes

,0
id_boleto,object
id_pagador,object
id_beneficiario,object
dt_emissao,datetime64[ns]
dt_vencimento,datetime64[ns]
dt_pagamento,datetime64[ns]
vlr_nominal,float64
vlr_baixa,float64
tipo_baixa,object
tipo_especie,object


In [ ]:
df_boletos_nuclea.head()

,id_boleto,id_pagador,id_beneficiario,dt_emissao,dt_vencimento,dt_pagamento,vlr_nominal,vlr_baixa,tipo_baixa,tipo_especie
0,00c6dec1b869d310010c8e3741262d7e96b0a622a7cf40...,24037765228ca1bce6743570728395541ca481ac79d854...,cf98a6d61719ae21047ea999c57051ffcae0809677068f...,2024-03-27,2024-05-09,2024-03-29,2714.66,NaN,5 - Baixa integral por solicitacao do cedente,DM DUPLICATA MERCANTIL
1,6548008e566d8f6696c5561a845248f61bfd2e3891711c...,24037765228ca1bce6743570728395541ca481ac79d854...,a711fb1183bc159fdcb672d61f9e052cda50f9e0a76e68...,2024-03-13,2024-05-05,2024-05-06,5522.63,5522.63,1 - Baixa integral intrabancaria,DMI DUPLICATA MERCANTIL INDICACAO
2,7409b9a978ccf059c68841c2317a3196adfd8d405854fa...,1c63664185066fb412e0803d751efbde86444efd8c265f...,8b4a7a67986a724fbc3ea10174aef547975c3d7257a3b1...,2024-05-17,2024-05-30,2024-05-27,3338.30,3338.30,0 - Baixa integral interbancaria,DM DUPLICATA MERCANTIL
3,2ef9ddc070a19a5af5d06c5605dc4fa90dc67052e70ffa...,1c63664185066fb412e0803d751efbde86444efd8c265f...,8b4a7a67986a724fbc3ea10174aef547975c3d7257a3b1...,2024-05-16,2024-05-29,2024-05-20,4519.18,4519.18,0 - Baixa integral interbancaria,DM DUPLICATA MERCANTIL
4,59e12a5e65532caf3bdb053f0e6d7af3b10a34b6c69029...,db8c5dc48c47d5ecae33324ee886afa21de700c19d8154...,00ac28a417bb5fe5daeafecd84190e8d794ed125971502...,2024-03-28,2024-05-27,2024-05-27,13416.18,13416.18,0 - Baixa integral interbancaria,DMI DUPLICATA MERCANTIL INDICACAO


In [ ]:
# Verificando se existe alguma linha duplicada
df_boletos_nuclea.duplicated().sum()

np.int64(0)

In [ ]:
# Verificando se existem dados nulos
df_boletos_nuclea.isnull().sum()

,0
id_boleto,0
id_pagador,0
id_beneficiario,0
dt_emissao,0
dt_vencimento,0
dt_pagamento,70
vlr_nominal,0
vlr_baixa,820
tipo_baixa,70
tipo_especie,0


In [ ]:
# Entendendo qual foi o tipo da baixa quando o valor da baixa é nulo, mas houve pagamento
df_boletos_nuclea.loc[
    (df_boletos_nuclea["vlr_baixa"].isnull()) & ~(df_boletos_nuclea["tipo_baixa"].isnull()),
    "tipo_baixa"
].unique()

array(['5 - Baixa integral por solicitacao do cedente',
       '8 - Baixa integral por solicitacao da instituicao destinataria',
       '7 - Baixa integral por decurso de prazo',
       '6 - Baixa integral por envio para protesto'], dtype=object)

In [ ]:
# Alimentação das colunas que possuem valores nulos
# Padronizamos que o valor da baixa será 0.0 nas colunas vlr_baixa e 'Não pago' nas colunas do tipo_baixa
df_boletos_nuclea["vlr_baixa"] = df_boletos_nuclea["vlr_baixa"].fillna(0.0)
df_boletos_nuclea["tipo_baixa"] = df_boletos_nuclea["tipo_baixa"].fillna("Não pago")

df_boletos_nuclea.isnull().sum()

,0
id_boleto,0
id_pagador,0
id_beneficiario,0
dt_emissao,0
dt_vencimento,0
dt_pagamento,70
vlr_nominal,0
vlr_baixa,0
tipo_baixa,0
tipo_especie,0


### DataFrame Auxiiar

In [ ]:
# Verificando o tipo dos dados do DF auxiliar
df_auxiliar_nuclea.dtypes

,0
id_cnpj,object
cd_cnae_prin,float64
uf,object
sacado_indice_liquidez_1m,float64
cedente_indice_liquidez_1m,float64
score_materialidade_evolucao,float64
media_atraso_dias,float64
indicador_liquidez_quantitativo_3m,float64
share_vl_inad_pag_bol_6_a_15d,float64
score_quantidade_v2,float64


In [ ]:
# Verificando se existe linha duplicada
df_auxiliar_nuclea.duplicated().sum()

np.int64(0)

In [ ]:
# Verificando se existe linha nula
df_auxiliar_nuclea.isnull().sum()

,0
id_cnpj,0
cd_cnae_prin,2
uf,359
sacado_indice_liquidez_1m,19
cedente_indice_liquidez_1m,2149
score_materialidade_evolucao,3
media_atraso_dias,5
indicador_liquidez_quantitativo_3m,20
share_vl_inad_pag_bol_6_a_15d,5
score_quantidade_v2,6


In [ ]:
# Quando o CNAE é nulo, não podemos fazer muita coisa com os dados, então removeremos
# Para o UF, colocaremos "ND"
df_auxiliar_nuclea["cd_cnae_prin"] = df_auxiliar_nuclea["cd_cnae_prin"].fillna(0000000)
df_auxiliar_nuclea["cd_cnae_prin"] = df_auxiliar_nuclea["cd_cnae_prin"].astype(int).astype(str)

df_auxiliar_nuclea["uf"] = df_auxiliar_nuclea["uf"].fillna("ND")
df_auxiliar_nuclea["uf"] = df_auxiliar_nuclea["uf"].str.strip().str.upper()

df_auxiliar_nuclea["score_quantidade_v2"] = df_auxiliar_nuclea["score_quantidade_v2"].fillna(0.0)
df_auxiliar_nuclea["score_materialidade_v2"] = df_auxiliar_nuclea["score_materialidade_v2"].fillna(0.0)

df_auxiliar_nuclea.isnull().sum()

,0
id_cnpj,0
cd_cnae_prin,0
uf,0
sacado_indice_liquidez_1m,19
cedente_indice_liquidez_1m,2149
score_materialidade_evolucao,3
media_atraso_dias,5
indicador_liquidez_quantitativo_3m,20
share_vl_inad_pag_bol_6_a_15d,5
score_quantidade_v2,0


### Criando colunas novas

In [ ]:
# Criando uma coluna de dias de atraso em boletos que foram pagos após o prazo de vencimento
df_boletos_nuclea["dias_atraso"] = (df_boletos_nuclea["dt_pagamento"] - df_boletos_nuclea["dt_vencimento"]).dt.days
df_boletos_nuclea["dias_atraso"] = df_boletos_nuclea["dias_atraso"].apply(lambda x: 0 if x < 0 else x)
df_boletos_nuclea.loc[df_boletos_nuclea["dt_pagamento"].isnull(), "dias_atraso"] = (datetime.now() - df_boletos_nuclea["dt_vencimento"]).dt.days
df_boletos_nuclea["dias_atraso"] = df_boletos_nuclea["dias_atraso"].astype(int)

df_boletos_nuclea.head()

,id_boleto,id_pagador,id_beneficiario,dt_emissao,dt_vencimento,dt_pagamento,vlr_nominal,vlr_baixa,tipo_baixa,tipo_especie,dias_atraso
0,00c6dec1b869d310010c8e3741262d7e96b0a622a7cf40...,24037765228ca1bce6743570728395541ca481ac79d854...,cf98a6d61719ae21047ea999c57051ffcae0809677068f...,2024-03-27,2024-05-09,2024-03-29,2714.66,0.00,5 - Baixa integral por solicitacao do cedente,DM DUPLICATA MERCANTIL,0
1,6548008e566d8f6696c5561a845248f61bfd2e3891711c...,24037765228ca1bce6743570728395541ca481ac79d854...,a711fb1183bc159fdcb672d61f9e052cda50f9e0a76e68...,2024-03-13,2024-05-05,2024-05-06,5522.63,5522.63,1 - Baixa integral intrabancaria,DMI DUPLICATA MERCANTIL INDICACAO,1
2,7409b9a978ccf059c68841c2317a3196adfd8d405854fa...,1c63664185066fb412e0803d751efbde86444efd8c265f...,8b4a7a67986a724fbc3ea10174aef547975c3d7257a3b1...,2024-05-17,2024-05-30,2024-05-27,3338.30,3338.30,0 - Baixa integral interbancaria,DM DUPLICATA MERCANTIL,0
3,2ef9ddc070a19a5af5d06c5605dc4fa90dc67052e70ffa...,1c63664185066fb412e0803d751efbde86444efd8c265f...,8b4a7a67986a724fbc3ea10174aef547975c3d7257a3b1...,2024-05-16,2024-05-29,2024-05-20,4519.18,4519.18,0 - Baixa integral interbancaria,DM DUPLICATA MERCANTIL,0
4,59e12a5e65532caf3bdb053f0e6d7af3b10a34b6c69029...,db8c5dc48c47d5ecae33324ee886afa21de700c19d8154...,00ac28a417bb5fe5daeafecd84190e8d794ed125971502...,2024-03-28,2024-05-27,2024-05-27,13416.18,13416.18,0 - Baixa integral interbancaria,DMI DUPLICATA MERCANTIL INDICACAO,0


In [ ]:
# Criando coluna nova de alvo de inadimplência, para nos ajudar com o ML
df_boletos_nuclea["alvo_inadimplencia"] = np.where(df_boletos_nuclea["dias_atraso"] > 0, 1, 0)
df_boletos_nuclea.head()

,id_boleto,id_pagador,id_beneficiario,dt_emissao,dt_vencimento,dt_pagamento,vlr_nominal,vlr_baixa,tipo_baixa,tipo_especie,dias_atraso,alvo_inadimplencia
0,00c6dec1b869d310010c8e3741262d7e96b0a622a7cf40...,24037765228ca1bce6743570728395541ca481ac79d854...,cf98a6d61719ae21047ea999c57051ffcae0809677068f...,2024-03-27,2024-05-09,2024-03-29,2714.66,0.00,5 - Baixa integral por solicitacao do cedente,DM DUPLICATA MERCANTIL,0,0
1,6548008e566d8f6696c5561a845248f61bfd2e3891711c...,24037765228ca1bce6743570728395541ca481ac79d854...,a711fb1183bc159fdcb672d61f9e052cda50f9e0a76e68...,2024-03-13,2024-05-05,2024-05-06,5522.63,5522.63,1 - Baixa integral intrabancaria,DMI DUPLICATA MERCANTIL INDICACAO,1,1
2,7409b9a978ccf059c68841c2317a3196adfd8d405854fa...,1c63664185066fb412e0803d751efbde86444efd8c265f...,8b4a7a67986a724fbc3ea10174aef547975c3d7257a3b1...,2024-05-17,2024-05-30,2024-05-27,3338.30,3338.30,0 - Baixa integral interbancaria,DM DUPLICATA MERCANTIL,0,0
3,2ef9ddc070a19a5af5d06c5605dc4fa90dc67052e70ffa...,1c63664185066fb412e0803d751efbde86444efd8c265f...,8b4a7a67986a724fbc3ea10174aef547975c3d7257a3b1...,2024-05-16,2024-05-29,2024-05-20,4519.18,4519.18,0 - Baixa integral interbancaria,DM DUPLICATA MERCANTIL,0,0
4,59e12a5e65532caf3bdb053f0e6d7af3b10a34b6c69029...,db8c5dc48c47d5ecae33324ee886afa21de700c19d8154...,00ac28a417bb5fe5daeafecd84190e8d794ed125971502...,2024-03-28,2024-05-27,2024-05-27,13416.18,13416.18,0 - Baixa integral interbancaria,DMI DUPLICATA MERCANTIL INDICACAO,0,0


## Alimentando o SQL

In [ ]:
# Código para alimentação dos dados Nuclea no Banco de Dados Oracle
USER = CREDENTIALS['user']
PASS = CREDENTIALS["password"]
DSN = CREDENTIALS["dsn"]  # Exemplo de DSN

# Criando a conexão
connection = None
try:
    with oracledb.connect(user=USER, password=PASS, dsn=DSN) as connection:
      cursor = connection.cursor()
      print("🔌Conectou")

      # Limpando as tabelas
      cursor.execute("TRUNCATE TABLE T_BF_BOLETO")
      cursor.execute("TRUNCATE TABLE T_BF_EMPRESA")
      print("🧹 Tabela limpa.")

      # EMPRESAS
      colunas_empresas_sql = ['id_cnpj', 'cd_cnae_prin', 'uf', 'score_materialidade_v2','score_quantidade_v2']
      dados_empresas = df_auxiliar_nuclea[colunas_empresas_sql].values.tolist()

      sql_insert_empresas = """
          INSERT INTO T_BF_EMPRESA (id_empresa, cd_cnae, sg_uf, vl_score_materialidade, vl_score_quantidade)
          VALUES (:1, :2, :3, :4, :5)
      """
      cursor.executemany(sql_insert_empresas, dados_empresas)
      print(f"   -> {cursor.rowcount} empresas inseridas.")

      # BOLETOS
      colunas_boletos_sql = [
          'id_boleto', 'id_pagador', 'id_beneficiario', 'vlr_nominal', 'vlr_baixa',
          'dt_emissao', 'dt_vencimento', 'dt_pagamento', 'tipo_baixa', 'dias_atraso', 'alvo_inadimplencia'
      ]
      dados_boletos = df_boletos_nuclea[colunas_boletos_sql].values.tolist()

      sql_insert_boletos = """
          INSERT INTO T_BF_BOLETO
          (id_boleto, id_pagador, id_beneficiario, vl_nominal, vl_baixa,
          dt_emissao, dt_vencimento, dt_pagamento, tp_baixa, nr_dias_atraso, vl_inadimplencia)
          VALUES (:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11)
      """
      cursor.executemany(sql_insert_boletos, dados_boletos)
      print(f"   -> {cursor.rowcount} boletos inseridos.")

      # Commit final
      connection.commit()
      print("\n✅ CARGA FINALIZADA COM SUCESSO!")

except oracledb.Error as e:
    print(f"❌ Erro fatal na conexão ou execução: {e}")

🔌Conectou
🧹 Tabela limpa.
   -> 4612 empresas inseridas.
   -> 7118 boletos inseridos.

✅ CARGA FINALIZADA COM SUCESSO!
